In [ ]:
%load_ext dotenv
%dotenv

import sqlite3
import pandas as pd
import os
import json
import hashlib
import pgOperation
from sqlalchemy.types import VARCHAR, Float, Integer, Date, Numeric
pd.options.mode.chained_assignment = None

ana = sqlite3.connect('./game_analysis.db')
pg = pgOperation.PgOperation('localhost', '5432', os.getenv('PG_USER'), '1', 'tfm')

In [ ]:
game_result_df = pg.readTable('game_results')
game_result_df['createtime'] = pd.to_datetime(game_result_df['createtime'])

In [ ]:
ods_main_games = pg.readTable('ods_main_games')
ods_main_games['prop'] = ods_main_games['prop'].apply(lambda x:eval(x))
pd.json_normalize(ods_main_games['prop'])

In [ ]:
tmp_game_df = pd.melt(
    # pd.concat([pd.DataFrame(pd.json_normalize(x)) for x in game_df['prop']],ignore_index=True)
    pd.json_normalize(ods_main_games['prop'])
    .reset_index(), id_vars='index', value_name='player_data').dropna(subset=['player_data']).set_index('index', drop=True)
flat_game_df = (pd.json_normalize(tmp_game_df['player_data']).join(tmp_game_df.reset_index(drop=False)['index']).set_index('index')) \
    .join(ods_main_games.drop(['seed_game_id', 'game_options', 'prop'], axis=1)).reset_index()
flat_game_df['position'] = flat_game_df.sort_index() \
            .groupby(['game_id']) \
            .cumcount() + 1
flat_game_df['rank'] = flat_game_df.sort_values(['playerScore'], ascending=[False]) \
            .groupby(['game_id']) \
            .cumcount() + 1

    # remove firstletter in flat_game_df['player'] if start with ~ or @
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x[1:] if x[0] in ['~', '@', '～'] else x)
# flat_game_df player to be lowercase
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x.lower())

flat_game_df.drop('index', axis=1).to_sql('ods_game_results', con=ana, if_exists='replace')

# remove firstletter in flat_game_df['player'] if start with ~ or @
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x[1:] if x[0] in ['~', '@', '～'] else x)
# flat_game_df player to be lowercase
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x.lower())

flat_game_df.drop('index', axis=1, inplace=True)
pg.writeDfToPg(flat_game_df, 'ods_game_results')
